In [ ]:
import numpy as np
import pandas as pd
labelsFarmacos = ["Presion","Azucar","Indice", "Alergia", 'Otras', 'Administrar']
labelsWeather = ['Tiempo', 'Temperatura', 'Humedad', 'Viento', 'Juega']

In [ ]:
farmacos_data = pd.read_csv("farmacos.csv",index_col=0)
weather_data = pd.read_csv("weather.csv",index_col=0)

In [ ]:
farmacos_data.head()

In [ ]:
weather_data.head()

In [ ]:
import pandas as pd
from sklearn import tree
from sklearn import preprocessing

data = pd.DataFrame()

le = preprocessing.LabelEncoder()



for v in labelsFarmacos:
  le.fit(farmacos_data[v])
  data[v] = le.transform(farmacos_data[v])

#construir el arbol
clf = tree.DecisionTreeClassifier(criterion = 'entropy',max_depth=4)

t_data = data[data.columns[0:4]]
t_respuesta = data[data.columns[4]]

#crear el modelo
clf = clf.fit(t_data, t_respuesta)

t_data,t_respuesta,clf

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file = None,
                                feature_names = list(t_data.columns.values),
                                class_names = ["No toma","Si toma"],
                                filled = True, rounded = True, special_characters = True)
                              
graph = graphviz.Source(dot_data)
graph

In [ ]:
import pandas as pd
import numpy as np
from sklearn import tree
#from sklearn import preprocessing

data = farmacos_data.copy()

data["Presion"] = np.where(data["Presion"] == "Alta",2,
                        np.where(data["Presion"] == "Media",1,0))

data["Azucar"] = np.where(data["Azucar"] == "Alto",1,0)

data["Indice"] = np.where(data["Indice"] == "Alto",1,0)
data["Alergia"] = np.where(data["Alergia"] == "Si",1,0)
data["Otras"] = np.where(data["Otras"] == "Si",1,0)
#data["Administrar"] = np.where(data["Administrar"] == "Si",1,0)

#construir el arbol
clf = tree.DecisionTreeClassifier(criterion = 'entropy',max_depth=4)

t_data = data[data.columns[0:4]]
t_respuesta = data[data.columns[4]]

#crear el modelo
clf = clf.fit(t_data, t_respuesta)

import graphviz
dot_data = tree.export_graphviz(clf, out_file = None,
                                feature_names = list(t_data.columns.values),
                                class_names = ["Administra","No administra"],
                                filled = True, rounded = True, special_characters = True)
                              
graph = graphviz.Source(dot_data)
graph

In [ ]:
import pandas as pd
from sklearn import tree
from sklearn import preprocessing

farmcos_test = pd.read_csv("farmacosTest.csv", index_col=0)


data = pd.DataFrame()
data_test = pd.DataFrame()

le = preprocessing.LabelEncoder()


for v in labelsFarmacos:
  le.fit(farmcos_test[v])
  data[v] = le.transform(farmcos_test[v])
  data_test[v] = le.transform(farmcos_test[v])

#construir el arbol
clf = tree.DecisionTreeClassifier(criterion = 'entropy',max_depth=4)

t_data = data[data.columns[0:4]]
t_respuesta = data[data.columns[4]]
x_test = data_test[data_test.columns[0:4]]


#crear el modelo
clf = clf.fit(t_data, t_respuesta)
y_respuesta = clf.predict(x_test)
t_data, t_respuesta, y_respuesta

In [ ]:
#calcular la entropia
def calcular_entropia(p):
  return -np.sum(p*np.log2(p))

#contar las clases de la caracteristica
valores, cuantos = np.unique(farmacos_data['Administrar'], return_counts=True)
entropia = calcular_entropia(cuantos/sum(cuantos))
print("Entropia del arbol",entropia)
valores,cuantos

In [ ]:
#calcular la ganancia
def calcular_ganancia(valores, var = 'Presion'):

  valores, cuantos = np.unique(farmacos_data[var], return_counts=True)

  info_a = []

  for e in valores:
    se = np.where(farmacos_data[var]==e)
    print("subconjunto",se)

    se_administra = farmacos_data['Administrar'][farmacos_data.index[se[0]]]
    print("Ejemplos del valor", se_administra)

    v_se_administra, c_se_quemadura = np.unique(se_administra,return_counts=True)
    print("subgrupos",v_se_administra,c_se_quemadura)

    #entropia del valor
    se_entropia = abs(calcular_entropia(c_se_quemadura/sum(c_se_quemadura)))
    print("entropia del valor",se_entropia)

    #probabilidad del valor
    se_size = sum(c_se_quemadura)/len(farmacos_data.index)
    print("probabilidad",se_size)

    info_a += [[se_entropia, se_size]]
    print("info_a",info_a)

  info_a = np.array(info_a)

  #calcular la entropia de la variable
  info_g = np.sum(info_a[:,0]*info_a[:,1])
  print("entropia de la variable",info_g)
  return info_g

info_a = calcular_ganancia(valores, var = 'Presion')
ganancia = entropia -info_a
print("ganancia de presion", ganancia)

In [ ]:
info_a = calcular_ganancia(valores, var = 'Azucar')
ganancia = entropia -info_a
print("ganancia de altura", ganancia)

In [ ]:
info_a = calcular_ganancia(valores, var = 'Indice')
ganancia = entropia -info_a
print("ganancia de peso", ganancia)

In [ ]:
info_a = calcular_ganancia(valores, var = 'Alergia')
ganancia = entropia -info_a
print("ganancia de Alergia", ganancia)

In [ ]:
from sklearn import tree
from sklearn import preprocessing

data = pd.DataFrame()

#Transformar a valores numeicos
le = preprocessing.LabelEncoder()

for v in labelsFarmacos:
    le.fit(farmacos_data[v])
    data[v] = le.transform(farmacos_data[v])

# Build tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

t_data = data[data.columns[0:4]]
t_target = data[data.columns[4]]

# Fit model to data
clf = clf.fit(t_data, t_target)
t_data,t_target,clf

# Visualize tree
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=list(t_data.columns.values),  
                         class_names=["No administra", "Administra"],
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

In [ ]:
data = farmacos_data.copy()

data["Pelo"] = np.where(data["Pelo"]=="rubio", 2, 
                          np.where(data["Pelo"]=="moreno", 1, 0))

data["Altura"] = np.where(data["Altura"]=="medio", 2, 
                               np.where(data["Altura"]=="alto", 1, 0))

data["Peso"] = np.where(data["Peso"]=="bajo", 2,
                        np.where(data["Peso"]=="medio",1,0))

data["Proteccion"] = np.where(data["Proteccion"]=="no", 0, 1)
    
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

t_data = data[data.columns[0:4]]
t_target = data[data.columns[4]]

clf = clf.fit(t_data, t_target)

import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=list(t_data.columns.values),  
                         class_names=["Don't Play", "Play"],  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph = graphviz.Source(dot_data)  
graph

In [ ]:
from sklearn import tree
from sklearn import preprocessing

data = pd.DataFrame()

#Transformar a valores numeicos
le = preprocessing.LabelEncoder()

var = ["Pelo", "Altura", "Peso", "Proteccion", "Quemadura"]

for v in var:
    le.fit(quemadura_data[v])
    data[v] = le.transform(quemadura_data[v])

# Build tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

t_data = data[data.columns[0:4]]
t_target = data[data.columns[4]]

# Fit model to data
clf = clf.fit(t_data, t_target)
t_data,t_target,clf

In [ ]:
from sklearn import tree
from sklearn import preprocessing

quemadura_test = pd.read_csv("arbQuemaduraTest.csv",index_col=0)
data = pd.DataFrame()
data1 = pd.DataFrame()
#Transformar a valores numeicos
le = preprocessing.LabelEncoder()

var = ["Pelo", "Altura", "Peso", "Proteccion", "Quemadura"]
for v in var:
    le.fit(quemadura_data[v])
    data[v] = le.transform(quemadura_data[v])
    data1[v] = le.transform(quemadura_test[v])

# Build tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

t_data = data[data.columns[0:4]]
t_target = data[data.columns[4]]

# Fit model to data
clf = clf.fit(t_data, t_target)
X_test = data1[data1.columns[0:4]]
y_pred = clf.predict(X_test)
y_pred


# Clima


In [ ]:
data_weather = pd.DataFrame()
le_w = preprocessing.LabelEncoder()

for v in labelsWeather:
    le_w.fit(weather_data[v])
    data_weather[v] = le_w.transform(weather_data[v])

#construir el arbol
clf_w = tree.DecisionTreeClassifier(criterion = 'entropy',max_depth=4)

t_data = data_weather[data_weather.columns[0:4]]
t_respuesta = data_weather[data_weather.columns[4]]

#crear el modelo
clf_w = clf_w.fit(t_data, t_respuesta)
t_data,t_respuesta,clf_w

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf_w, out_file=None,
                         feature_names=list(t_data.columns.values),
                         class_names=["No Juega", "Si Juega"],
                         filled=True, rounded=True,
                         special_characters=True)
graph = graphviz.Source(dot_data)
graph

In [ ]:
from sklearn import tree

data_weather = weather_data.copy()
data_weather["Tiempo"] = np.where(data_weather["Tiempo"]=="Soleado", 2,
                          np.where(data_weather["Tiempo"]=="Nubloso", 1, 0))
data_weather["Temperatura"] = np.where(data_weather["Temperatura"]=="Alta", 2,
                          np.where(data_weather["Temperatura"]=="Media", 1, 0))
data_weather["Humedad"] = np.where(data_weather["Humedad"] == "Alta",1,0)
data_weather["Viento"] = np.where(data_weather["Humedad"] == "Si",1,0)


#construir el arbol
clf = tree.DecisionTreeClassifier(criterion = 'entropy',max_depth=4)

t_data = data_weather[data_weather.columns[0:4]]
t_respuesta = data_weather[data_weather.columns[4]]

#crear el modelo
clf = clf.fit(t_data, t_respuesta)

dot_data = tree.export_graphviz(clf, out_file=None,
                         feature_names=list(t_data.columns.values),
                         class_names=["No Juega", "Si Juega"],
                         filled=True, rounded=True,
                         special_characters=True)
graph = graphviz.Source(dot_data)
graph



In [ ]:
weather_test = pd.read_csv("weatherTest.csv",index_col=0)
data_weather = pd.DataFrame()
data_weather_test = pd.DataFrame()

le = preprocessing.LabelEncoder()

for v in labelsWeather:
    le.fit(weather_data[v])
    data_weather[v] = le.transform(weather_data[v])
    data_weather_test[v] = le.transform(weather_test[v])

# Build tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

t_data = data_weather[data_weather.columns[0:4]]
t_target = data_weather[data_weather.columns[4]]
w_test = data_weather_test[data_weather_test.columns[0:4]]

# Fit model to data
clf = clf.fit(t_data, t_target)
y_pred = clf.predict(w_test)
t_data, t_target, y_pred


In [ ]:
# Calculate entropy

val, counts = np.unique(weather_data['Juega'], return_counts=True)
entropy_weather = calcular_entropia(counts/counts.sum())
print("Entropia del arbol Clima:", entropy_weather.round(4))
val, counts

In [ ]:
# Calculate profit
def calculate_profit(var = 'Tiempo'):
    values, counts = np.unique(weather_data[var], return_counts=True)
    inf_a = []

    for e in values:
        se = np.where(weather_data[var] == e)
        print("Subconjunto ",se)

        se_juega = weather_data['Juega'][weather_data.index[se[0]]]
        print("Ejemplos de valor:", se_juega)

        v_se_juega, c_se_juega_quemadura = np.unique(se_juega, return_counts=True)
        print("Subgrupos:", v_se_juega, c_se_juega_quemadura)

        se_entropy = abs(calcular_entropia(c_se_juega_quemadura/c_se_juega_quemadura.sum()))
        print("Entropia valor:", se_entropy)

        se_size_weather = sum(c_se_juega_quemadura)/len(weather_data.index)
        print("Probabilidad:", se_size_weather)

        inf_a += [[se_entropy, se_size_weather]]
        print("inf_a", inf_a)
    inf_a = np.array(inf_a)

    inf_g = np.sum(inf_a[:,0]*inf_a[:,1])
    print("Entropia general:", inf_g)
    return inf_g
var = 'Tiempo'
inf_a = calculate_profit(var)
profit = entropy_weather - inf_a
print("Beneficio de",var,": ", profit.round(4))

In [ ]:
var = 'Temperatura'
inf_a = calculate_profit(var)
profit = entropy_weather - inf_a
print("Beneficio de",var,": ", profit.round(4))

In [ ]:
var = 'Humedad'
inf_a = calculate_profit(var)
profit = entropy_weather - inf_a
print("Beneficio de",var,": ", profit.round(4))

In [ ]:
var = 'Viento'
inf_a = calculate_profit(var)
profit = entropy_weather - inf_a
print("Beneficio de",var,": ", profit.round(4))